In [1]:
import pandas as pd

# Load datasets
train_data = pd.read_csv('train.csv')
x_test = pd.read_csv('x_test.csv')
sample_submission = pd.read_csv('sample_submission.csv')

# Display first few rows of each dataset
train_data.head(), x_test.head(), sample_submission.head()

from sklearn.preprocessing import StandardScaler

# Extracting features and target from the training data
X_train = train_data.drop(columns=['id', 'y'])
y_train = train_data['y']

# Initializing the scaler
scaler = StandardScaler()

# Fitting the scaler on the training data and transforming both training and test data
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(x_test.drop(columns=['id']))

X_train_scaled[:5], X_test_scaled[:5]  # Display first 5 rows of scaled data for both training and test sets

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Initialize the linear regression model
lr_model = LinearRegression()

# Train the model
lr_model.fit(X_train_scaled, y_train)

# Predict on training data for evaluation
y_train_pred = lr_model.predict(X_train_scaled)

# Calculate the MSE on training data
mse_train = mean_squared_error(y_train, y_train_pred)

mse_train



1710.6670090547075

In [2]:
from sklearn.linear_model import Ridge, Lasso

# Initialize Ridge and Lasso models
ridge_model = Ridge()
lasso_model = Lasso()

# Train the models
ridge_model.fit(X_train_scaled, y_train)
lasso_model.fit(X_train_scaled, y_train)

# Predict on training data for evaluation
y_train_pred_ridge = ridge_model.predict(X_train_scaled)
y_train_pred_lasso = lasso_model.predict(X_train_scaled)

# Calculate the MSE on training data for both models
mse_train_ridge = mean_squared_error(y_train, y_train_pred_ridge)
mse_train_lasso = mean_squared_error(y_train, y_train_pred_lasso)

mse_train_ridge, mse_train_lasso


(1813.7723061914548, 2031.209448053145)

In [4]:
from sklearn.model_selection import GridSearchCV

# Define the hyperparameters to search
param_grid = {
    'alpha': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]
}

# Grid search with cross-validation for Ridge regression
ridge_search = GridSearchCV(Ridge(), param_grid, cv=5, scoring='neg_mean_squared_error')
ridge_search.fit(X_train_scaled, y_train)

# Grid search with cross-validation for Lasso regression
lasso_search = GridSearchCV(Lasso(), param_grid, cv=5, scoring='neg_mean_squared_error')
lasso_search.fit(X_train_scaled, y_train)

# Get the best parameters and scores for both models
best_alpha_ridge = ridge_search.best_params_['alpha']
best_mse_ridge = -ridge_search.best_score_

best_alpha_lasso = lasso_search.best_params_['alpha']
best_mse_lasso = -lasso_search.best_score_

best_alpha_ridge, best_mse_ridge, best_alpha_lasso, best_mse_lasso

# Predict on the test data using the best Lasso model
y_test_pred = lasso_search.best_estimator_.predict(X_test_scaled)

# Preparing the submission dataframe
submission = x_test[['id']].copy()
submission['y'] = y_test_pred

submission.head()



/Users/mihir/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.484e+04, tolerance: 7.976e+01
  model = cd_fast.enet_coordinate_descent(
/Users/mihir/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.813e+04, tolerance: 7.245e+01
  model = cd_fast.enet_coordinate_descent(
/Users/mihir/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duali

,id,y
0,160,107.887927
1,161,227.246385
2,162,138.843740
3,163,176.657423
4,164,149.088094


In [5]:
from sklearn.ensemble import GradientBoostingRegressor

# Initialize the gradient boosting regressor
gb_model = GradientBoostingRegressor(n_estimators=100, random_state=42)

# Train the model
gb_model.fit(X_train_scaled, y_train)

# Predict on training data for evaluation
y_train_pred_gb = gb_model.predict(X_train_scaled)

# Calculate the MSE on training data
mse_train_gb = mean_squared_error(y_train, y_train_pred_gb)

mse_train_gb


71.12658100462362

In [6]:
from sklearn.model_selection import cross_val_score

# Perform cross-validation on the Gradient Boosting Regressor
cv_scores = cross_val_score(gb_model, X_train_scaled, y_train, cv=5, scoring='neg_mean_squared_error')

# Calculate mean and standard deviation of cross-validation scores
mean_mse_gb_cv = -cv_scores.mean()
std_mse_gb_cv = cv_scores.std()

mean_mse_gb_cv, std_mse_gb_cv


(4442.096981260921, 510.3552826913761)

In [9]:
# Define hyperparameters for grid search
param_grid_gb = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.05, 0.1, 0.5],
    'max_depth': [3, 4, 5, 6]
}

# Grid search with cross-validation for Gradient Boosting Regressor
gb_search = GridSearchCV(GradientBoostingRegressor(random_state=42), param_grid_gb, cv=5, scoring='neg_mean_squared_error')
gb_search.fit(X_train_scaled, y_train)

# Get the best parameters and score
best_params_gb = gb_search.best_params_
best_mse_gb = -gb_search.best_score_

print(best_params_gb, best_mse_gb)

# Predict on the test data using the best Gradient Boosting Regressor from randomized search
y_test_pred_gb = gb_search.best_estimator_.predict(X_test_scaled)

# Update the submission dataframe with new predictions
submission['y'] = y_test_pred_gb

submission.head()

# Save the submission dataframe to a CSV file
submission_file_path = "submissions_test.csv"
submission.to_csv(submission_file_path, index=False)

submission_file_path




{'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 200} 4120.996225188528


'submissions_test.csv'